<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download and unzip (run only once!).
PATH_TO_LEFT_IMAGES = "leftImg8bit/train_extra/schweinfurt/"
PATH_TO_DISPARITY = "disparity/train_extra/schweinfurt/"
PATH_TO_CAMERA = "camera/train_extra/schweinfurt/"
PATH_TO_VEHICLE = "vehicle/train_extra/schweinfurt/"

# Remove data directory if it is already loaded (if needed).
#!rm -r data
# Login.
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=ftaubner@ethz.ch&password=semantic_dudes&submit=Login' https://www.cityscapes-dataset.com/login/
# Get left camera images. I put it on OneDrive so that only Schweinfurt has to be downloaded.
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199734&authkey=AC5K24PFcrSPFl4" -O leftImg8bit_trainextra.zip
# Alternatively, it can be downloaded from CityScapes website directly:
#   (!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=4)
# Extract.
!mkdir data
!unzip -qq leftImg8bit_trainextra.zip "$PATH_TO_LEFT_IMAGES"* -d data
# And delete.
!rm leftImg8bit_trainextra.zip

# Get disparity maps.
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199735&authkey=AEwGRlH_TySnyPM" -O disparity_trainextra.zip
# Original file:
#   (!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=22)
!unzip -qq disparity_trainextra.zip "$PATH_TO_DISPARITY"* -d data
!rm disparity_trainextra.zip
# Get camera intrinsics. 
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=9
!unzip -qq camera_trainextra.zip "$PATH_TO_CAMERA"* -d data
!rm camera_trainextra.zip
# Get vehicle odometry.
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=11
!unzip -qq vehicle_trainextra.zip "$PATH_TO_VEHICLE"* -d data
!rm vehicle_trainextra.zip

import os
PATH_TO_LEFT_IMAGES = os.path.join("data/", PATH_TO_LEFT_IMAGES)
PATH_TO_DISPARITY = os.path.join("data/", PATH_TO_DISPARITY)
PATH_TO_CAMERA = os.path.join("data/", PATH_TO_CAMERA)
PATH_TO_VEHICLE = os.path.join("data/", PATH_TO_VEHICLE)

print("Finished unzipping.")

--2020-03-25 20:12:04--  https://www.cityscapes-dataset.com/login/
Resolving www.cityscapes-dataset.com (www.cityscapes-dataset.com)... 139.19.217.8
Connecting to www.cityscapes-dataset.com (www.cityscapes-dataset.com)|139.19.217.8|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.cityscapes-dataset.com/downloads/ [following]
--2020-03-25 20:12:06--  https://www.cityscapes-dataset.com/downloads/
Reusing existing connection to www.cityscapes-dataset.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html’

index.html              [   <=>              ]  42.77K  79.3KB/s    in 0.5s    

2020-03-25 20:12:09 (79.3 KB/s) - ‘index.html’ saved [43792]

--2020-03-25 20:12:10--  https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199734&authkey=AC5K24PFcrSPFl4
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com

In [2]:

# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version
# opencv is pre-installed on colab


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 723.9MB 25kB/s 
     |████████████████████████████████| 4.1MB 46.4MB/s 
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
  Found existing installation: torchvision 0.5.0
    Uninstalling torchvision-0.5.0:
      Successfully uninstalled torchvision-0.5.0
     |████████████████████████████████| 276kB 2.5MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=54cdfa413827d9a3e1fcc17bfb2d400ce0673692c601a2a9d8880adc516f078d
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-72_ssaxl
  Running command git 

In [3]:

# install detectron2:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html
     |████████████████████████████████| 6.2MB 818kB/s 
  Created wheel for fvcore: filename=fvcore-0.1.dev200325-cp36-none-any.whl size=38930 sha256=0fdc1b8259666cb340001b88bc1bd2e02f2de11e276dae80cf254b1afdf6a670
  Stored in directory: /root/.cache/pip/wheels/58/4f/d1/6b0ada294145df5fe73092d9849d27be5b5b720fd8cfdd1b98
Successfully built fvcore


In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [0]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
               
# Manually picked some static classes


static_classes = ['other', 'traffic light', 'fire hydrant', 
                  'stop sign', 'parking meter', 'bench']

In [6]:
# Create dict that maps coco classes to classes of interest
coco_to_sel_map = {}
for class_name in class_names:
  if class_name in static_classes:
    coco_to_sel_map[class_name] = class_name
  else:
    coco_to_sel_map[class_name] = 'other'

print(coco_to_sel_map)



{'BG': 'other', 'person': 'other', 'bicycle': 'other', 'car': 'other', 'motorcycle': 'other', 'airplane': 'other', 'bus': 'other', 'train': 'other', 'truck': 'other', 'boat': 'other', 'traffic light': 'traffic light', 'fire hydrant': 'fire hydrant', 'stop sign': 'stop sign', 'parking meter': 'parking meter', 'bench': 'bench', 'bird': 'other', 'cat': 'other', 'dog': 'other', 'horse': 'other', 'sheep': 'other', 'cow': 'other', 'elephant': 'other', 'bear': 'other', 'zebra': 'other', 'giraffe': 'other', 'backpack': 'other', 'umbrella': 'other', 'handbag': 'other', 'tie': 'other', 'suitcase': 'other', 'frisbee': 'other', 'skis': 'other', 'snowboard': 'other', 'sports ball': 'other', 'kite': 'other', 'baseball bat': 'other', 'baseball glove': 'other', 'skateboard': 'other', 'surfboard': 'other', 'tennis racket': 'other', 'bottle': 'other', 'wine glass': 'other', 'cup': 'other', 'fork': 'other', 'knife': 'other', 'spoon': 'other', 'bowl': 'other', 'banana': 'other', 'apple': 'other', 'sandwic

In [10]:
!mkdir data/Results

DATA_DIR="data/leftImg8bit/train_extra/schweinfurt"
SAVE_DIR="data/Results"

model_file="COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file(model_file))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_file)

predictor = DefaultPredictor(cfg)

import glob
import os
import json

images_path = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
n_images = len(images_path)
count = 0
# create empty dictionary 
features = {}
# create empty list of the results
features['results'] = []

for image_path in images_path:
  # Load image from path
  image = cv2.imread(image_path)
  # Predict bounding boxes and classes
  results = predictor(image)
  print("Image number:{} of {} is beeing processed".format(count+1, n_images))

  # Save visualization in path DATA_DIR/Results with same name as images
  v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
  v = v.draw_instance_predictions(results["instances"].to("cpu"))
  head, tail = os.path.split(image_path)
  name = os.path.join(SAVE_DIR,"{}".format(tail))
  cv2.imwrite(name, v.get_image()[:, :, ::-1])

  # export classes and bounding boxes
  classes_tensor = results["instances"].pred_classes
  boxes_tensor = results["instances"].pred_boxes
  classes = classes_tensor.cpu().tolist()
  box = boxes_tensor.to("cpu").tensor.tolist()
  static_class_ids = []
  static_boxes = []
  i = 0
  for class_id in classes:
    if coco_to_sel_map[class_names[class_id]] != 'other':
      static_class_ids.append(static_classes.index(coco_to_sel_map[class_names[class_id]]))
      static_boxes.append(box[i])
      i += 1 
  # put them in a dicitonary to later save them into a json file
  features['results'].append({'image_id': tail, 'classes': static_class_ids, 'boxes': static_boxes})
  count += 1

mkdir: cannot create directory ‘data/Results’: File exists
Image number:1 of 710 is beeing processed
Image number:2 of 710 is beeing processed
Image number:3 of 710 is beeing processed
Image number:4 of 710 is beeing processed
Image number:5 of 710 is beeing processed
Image number:6 of 710 is beeing processed
Image number:7 of 710 is beeing processed
Image number:8 of 710 is beeing processed
Image number:9 of 710 is beeing processed
Image number:10 of 710 is beeing processed
Image number:11 of 710 is beeing processed
Image number:12 of 710 is beeing processed
Image number:13 of 710 is beeing processed
Image number:14 of 710 is beeing processed
Image number:15 of 710 is beeing processed
Image number:16 of 710 is beeing processed
Image number:17 of 710 is beeing processed
Image number:18 of 710 is beeing processed
Image number:19 of 710 is beeing processed
Image number:20 of 710 is beeing processed
Image number:21 of 710 is beeing processed
Image number:22 of 710 is beeing processed
Imag

In [9]:
print(static_class_ids)

[]


In [0]:
# Write to json file
import json
# format: row: image_id , [class_ids], [boxes] , with boxes as x1, y1, x2, 
FEATURE_FILE = 'results.json'
with open(FEATURE_FILE, 'w') as outfile:
    json.dump(features, outfile)

In [0]:
a = [1, 2, 3, 4]
if 1 !=1:
  print(coco_to_sel_map['donut'])
